# Compare accuracy of ICET with different distortion rectification methods  

In [1]:
from vedo import *
import os
from ipyvtklink.viewer import ViewInteractiveWidget
import pykitti
import numpy as np
import tensorflow as tf
import time
import pickle

#limit GPU memory ------------------------------------------------
gpus = tf.config.experimental.list_physical_devices('GPU')
print(gpus)
if gpus:
  try:
    memlim = 4*1024
    tf.config.experimental.set_virtual_device_configuration(gpus[0], [tf.config.experimental.VirtualDeviceConfiguration(memory_limit=memlim)])
  except RuntimeError as e:
    print(e)
#-----------------------------------------------------------------

from tensorflow.math import sin, cos, tan
import tensorflow_probability as tfp
import sys
current = os.getcwd()
parent_directory = os.path.dirname(current)
sys.path.append(parent_directory)
from ICET_spherical import ICET
from utils import R_tf
from metpy.calc import lat_lon_grid_deltas
from pioneer.das.api.platform import Platform
from scipy.spatial.transform import Rotation as R
from pioneer.das.api.egomotion.imu_egomotion_provider import IMUEgomotionProvider as emp 
from matplotlib import pyplot as plt

%load_ext autoreload
%autoreload 2
%autosave 180
%matplotlib notebook

2023-08-09 12:14:32.423647: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-08-09 12:14:32.518804: E tensorflow/stream_executor/cuda/cuda_blas.cc:2981] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2023-08-09 12:14:32.894191: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer.so.7'; dlerror: libnvinfer.so.7: cannot open shared object file: No such file or directory; LD_LIBRARY_PATH: /home/derm/anaconda3/envs/py39/lib/python3.9/site-packages/cv2/../../lib64:
2023-08-09 12:14:32.894249: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer_p

[PhysicalDevice(name='/physical_device:GPU:0', device_type='GPU')]


2023-08-09 12:14:33.541405: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-08-09 12:14:33.542188: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:980] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2023-08-09 12:14:33.542402: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:980] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2023-08-09 12:14:33.542545: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:980] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zer

ImportError: /home/derm/anaconda3/envs/py39/lib/python3.9/site-packages/open3d/linux/open3d.so: undefined symbol: _Py_ZeroStruct

# Motion distortion in simple box scene

In [2]:
#case2: forward linear motion 
# scan1 = np.load("sample_data/paper_figures/case2/raw_frame_2.npy") #static
scan1 = np.load("sample_data/paper_figures/case2/raw_frame_3.npy")
scan2 = np.load("sample_data/paper_figures/case2/rectified_linear_frame_3.npy")
#erase the extra points in 2nd scan that didn't get properly rectified
scan2 = scan2[:-150,:]

#remove ground plance from both scans
# scan1 = scan1[scan1[:,2] > -1.2]
# scan2 = scan2[scan2[:,2] > -1.2]

it = ICET(cloud1 = scan1, cloud2 = scan2, fid = 50, niter = 10, 
           draw = True, group = 2, RM = False, DNN_filter = False)#, x0 = initial_guess)
print("\n predicted standard deviations of error: \n", it.pred_stds)
ViewInteractiveWidget(it.plt.window)


2023-06-23 09:56:57.980334: I tensorflow/core/platform/default/subprocess.cc:304] Start cannot spawn child process: No such file or directory
/home/derm/anaconda3/envs/py39/lib/python3.9/site-packages/vtkmodules/util/numpy_support.py:74: DeprecationWarning: `np.bool` is a deprecated alias for the builtin `bool`. To silence this warning, use `bool` by itself. Doing this will not modify any behavior and is safe. If you specifically wanted the numpy scalar type, use `np.bool_` here.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations
  _vtk_np = {vtkConstants.VTK_BIT:numpy.bool,
2023-06-23 09:56:59.873695: I tensorflow/stream_executor/cuda/cuda_blas.cc:1614] TensorFloat-32 will be used for the matrix multiplication. This will only be logged once.
2023-06-23 09:56:59.937402: I tensorflow/core/util/cuda_solvers.cc:179] Creating GpuSolver handles for stream 0x8397770


iter 0
iter 1
iter 2
iter 3
iter 4
iter 5
iter 6
iter 7
iter 8
iter 9

 estimated solution vector X: 
 tf.Tensor(
[-2.7701050e-01  3.9676409e-03 -6.7204070e-05 -1.7952749e-05
 -3.8813214e-05  5.3816740e-03], shape=(6,), dtype=float32)

 predicted standard deviations of error: 
 tf.Tensor(
[4.7771272e-04 2.2880972e-04 3.7311263e-05 7.4056293e-06 8.1525232e-06
 3.9897026e-05], shape=(6,), dtype=float32)


ViewInteractiveWidget(height=1043, layout=Layout(height='auto', width='100%'), width=1280)

# Linear motion through simulated urban area

In [3]:
### 10 mph -- test1
scan1 = np.load("sample_data/test1/rectified_linear_frame_4.npy")
scan2 = np.load("sample_data/test1/rectified_linear_frame_5.npy")

# # simple linear motion -- test2
# scan1 = np.load("sample_data/test2/rectified_linear_frame_2.npy")
# scan2 = np.load("sample_data/test2/rectified_linear_frame_3.npy")
# initial_guess = tf.constant([1.25, 0, 0, 0, 0, 0])

it = ICET(cloud1 = scan1, cloud2 = scan2, fid = 110, niter = 10, 
           draw = True, group = 2, RM = False, DNN_filter = False)#, x0 = initial_guess)
print("\n predicted standard deviations of error: \n", it.pred_stds)
ViewInteractiveWidget(it.plt.window)


iter 0
iter 1
iter 2
iter 3
iter 4
iter 5
iter 6
iter 7
iter 8
iter 9

 estimated solution vector X: 
 tf.Tensor(
[ 4.4003969e-01  2.8940639e-04 -1.0319555e-04  4.3787759e-07
 -2.2514860e-05  1.3771109e-04], shape=(6,), dtype=float32)

 predicted standard deviations of error: 
 tf.Tensor(
[4.1836404e-04 2.2216886e-04 7.5533426e-05 1.1627386e-05 1.6026030e-05
 2.6369074e-05], shape=(6,), dtype=float32)


ViewInteractiveWidget(height=1043, layout=Layout(height='auto', width='100%'), width=1280)

In [4]:
#load ground truth data
gt = np.load("sample_data/test1/base_vel_7.npy")
print(len(gt))

1001


# Generate relative distortion figures


In [3]:
# fn1 = "/home/derm/ASAR/v3/point_cloud_rectification/sample_data/paper_figures/relative_distortion_fig/raw_frame_1.npy"
# fn2 = "/home/derm/ASAR/v3/point_cloud_rectification/sample_data/paper_figures/relative_distortion_fig/raw_frame_3.npy"

fn1 = "/home/derm/ASAR/v3/point_cloud_rectification/sample_data/paper_figures/relative_distortion_fig/raw_frame_v2_1.npy"
fn2 = "/home/derm/ASAR/v3/point_cloud_rectification/sample_data/paper_figures/relative_distortion_fig/raw_frame_v2_3.npy"

pc1 = np.load(fn1)
pc2 = np.load(fn2)

# #manually distort pc2
# pc2 = np.zeros(np.shape(pc1))
# pc2[:,:] = pc1[:,:]
# # pc2 = pc2[:len(pc2)//2,:]
# stretchvec = np.linspace(1,1.1,len(pc1))
# print(stretchvec)
# pc2[:,0] = pc2[:,0] * stretchvec

# #remove points at the beginning of scan 2 (Debug)
# pc2[-1000:,0][pc2[-1000:,2] > -1.5] += 6

#remove inf values 
pc1[pc1>10000] = 0
pc1[pc1<-10000] = 0
pc2[pc2>10000] = 0
pc2[pc2<-10000] = 0

# translate scan2 to align with 1
pc2[:,0] += 3.5

#remove ground planes
pc1 = pc1[pc1[:,2] > -1.5]
pc2 = pc2[pc2[:,2] > -1.5]

plt = Plotter(N = 1, axes = 4, bg = (1, 1, 1), interactive = True) #axes = 4 (simple), 1(scale)
settings.useParallelProjection = True #makes view orthographic,
disp=[]

alph = 1
radius = 2
redCloud = Points(pc1, c = "#CB2314", r = radius, alpha = alph).renderPointsAsSpheres(False)
disp.append(redCloud)
blueCloud = Points(pc2, c = "#2c7c94", r = radius, alpha = alph).renderPointsAsSpheres(False)
disp.append(blueCloud)


plt.show(disp, "Relative Distortion Example")
ViewInteractiveWidget(plt.window)

ViewInteractiveWidget(height=1043, layout=Layout(height='auto', width='100%'), width=1280)